In [1]:
from ABDB import database as db
import numpy as np
from rich.progress import track
import copy
from sklearn.model_selection import train_test_split
import torch
from einops import rearrange
import json

/Users/fabian/miniconda3/envs/auto-db-pipeline/lib/python3.9/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


### 1. Data preparation
**1.1 Get data from SAbDab**

Extract CDR sequencs and coordinate of backbone atoms from antibodies in SAbDab. This steps takes approximately 2 hrs, thus the output is saved in .npy file

In [2]:
# dictionaries to convert one letter, three letter and numerical amino acid codes
aa1 = "ACDEFGHIKLMNPQRSTVWY"
aa3 = ["ALA", "CYS", "ASP", "GLU", "PHE", "GLY", "HIS", "ILE", "LYS", "LEU", "MET", "ASN", "PRO", "GLN", "ARG", "SER",
       "THR", "VAL", "TRP", "TYR", ]

short2long = {}
long2short = {}
short2num = {}

for ind in range(0, 20):
    long2short[aa3[ind]] = aa1[ind]
    short2long[aa1[ind]] = aa3[ind]
    short2num[aa1[ind]] = ind

In [3]:
# functions to filter entries in SAbDAb

def filter_abs(pdb_list):
    '''
    Filter a list of PDB ids obtained from SAbDab and removes FABS where one of the chains is missing or where
    heavy and light chains have the same name.
    '''
    filtered_list = []
    i = 0
    
    for pdb in track(pdb_list, description='Filter FABs'):
        i += 1
        fab = db.fetch(pdb).fabs[0]

        if fab.VH == fab.VL:
            continue
        elif fab.VH == 'NA' or fab.VL == 'NA':
            continue
        else:
            filtered_list.append(pdb)

    return filtered_list

In [4]:
# functions to extract and format relevant data from a SAbDab FAB. Given a FAB two dictionaries are returned for CDR and anchor 
# sequences and thier backbone coordinates

def split_structure_in_regions(fab):
    '''
    Split FAB into regions.

    Takes FAB as input an returns a dictionary with keys: regions, values: residues in region
    regions = ['fwh1', 'cdrh1', 'fwh2', 'cdrh2', 'fwh3', 'cdrh3', 'fwh4', 'fwl1', 'cdrl1', 'fwl2', 'cdrl2', 'fwl3', 'cdrl3', 'fwl4']
    '''
    ab_regions = dict()
    struc = fab.get_structure()

    for chain in [fab.VH, fab.VL]:

        # Chian.get_residues() is a generator that loops through residue
        for residue in struc[chain].get_residues():

            # residue.region indicates in which cdr or framework region the residue is
            if residue.region in ab_regions:
                ab_regions[residue.region].append(residue)
            else:
                ab_regions[residue.region] = [residue]

    return ab_regions

def get_slice(ab_regions, CDR):
    '''
    Returns a slice of residues containing a CDR plus two anchor residues on each side,
    given a FAB split in to regions and a spefied CDR.
    '''
    chain = CDR[0].lower()
    loop = CDR[1]

    slice = ab_regions['fw' + chain + loop][-2:]
    slice += ab_regions['cdr' + chain + loop]
    slice += ab_regions['fw' + chain + str(int(loop) + 1 )][:2]

    return slice

def cdr_anchor_seq(ab_regions, CDR):
    '''
    Retruns sequence of a CDR plus two anchors on each side,
    given a FAB split in to regions and a spefied CDR.
    '''
    slice = get_slice(ab_regions, CDR)
    CDR_seq = []

    for res in slice:
        CDR_seq.append(long2short[res.resname])

    return CDR_seq

def cdr_anchor_BB_coord(ab_regions, CDR):
    '''
    Returns coordinates of backbone atoms of a CDR plus two anchors on each side,
    given a FAB split in to regions and a spefied CDR.
    '''
    slice = get_slice(ab_regions, CDR)
    CDR_BB_coord = np.zeros((len(slice), 4, 3))
    BB_atoms = ["CA", "C", "N", "CB"]

    for i in range(len(slice)):
        res = slice[i]
        for j in range(len(BB_atoms)):
            atom = BB_atoms[j]

            # if residue is glycine use CA coordinates for CB
            if res.resname == 'GLY' and atom == 'CB':
                atom = "CA"
                
            coord = res[atom].coord

            CDR_BB_coord[i, j, :] = coord
    
    return CDR_BB_coord

def get_cdr_anchor_seqs(ab_regions, CDRs = ["H1", "H2", "H3", "L1", "L2", "L3"]):
    '''
    Get sequences of all CDRs in a FAB.

    Returns a dictionary with keys: CDRs, values: CDR + anchor sequence
    '''
    CDR_seqs = dict()

    for CDR in CDRs:
        CDR_seqs[CDR] = cdr_anchor_seq(ab_regions, CDR)
   
    return CDR_seqs

def get_cdr_anchor_BB_coords(cdr_residues, CDRs = ["H1", "H2", "H3", "L1", "L2", "L3"]):
    '''
    Get backbone coordinates of all CDRs in a FAB.
    
    Returns a dictionary with keys: CDRs, values: CDR + anchor backbone coordinates
    '''
    CDR_BB_coords = dict()

    for CDR in CDRs:
        CDR_BB_coords[CDR] = cdr_anchor_BB_coord(cdr_residues, CDR)
   
    return CDR_BB_coords

In [6]:
# function to retrieve PDB ids from SAbDab and runs functions in above cells for each individual FAB.

def get_sabdab_fabs(pdb_list):
    '''
    Get fabs from sabdab given a list of pdbs, extracts CDR sequences and coordinates and formats the data for the next steps.

    returns CDR_seqs: list of dictionaries,
                      each dictionary contains data of one FAB, keys: CDR, value: CDR sequence
    returns CDR_BB_coords: list of dictionaries,
                           each dictionary contains data of one FAB, keys: CDR, value: CDR backbone coordinates
    '''
    pdb_list = filter_abs(pdb_list)

    CDR_seqs = list()
    CDR_BB_coords = list()

    for pdb_id in track(pdb_list, description='Load data from SAbDab'):
        pdb = db.fetch(pdb_id)
        for fab in pdb.fabs:
            try: # some fab have errors and throw exceptions, ignore these
                ab_regions = split_structure_in_regions(fab)
                cdr_seqs = get_cdr_anchor_seqs(ab_regions)
                cdr_BB_coords = get_cdr_anchor_BB_coords(ab_regions)

                CDR_seqs.append(cdr_seqs)
                CDR_BB_coords.append(cdr_BB_coords)
            except Exception:
                pass

    return CDR_seqs, CDR_BB_coords

Code that runs the above functions to download all FABs from SAbDab and extract data about CDR and anchor sequences and backbone coordinates. Data is saved in a .npy file.

In [7]:
# use imgt numbering
db.set_numbering_scheme("imgt")
db.set_region_definition("imgt")

# list of all pdb ids in SAbDab
all_pdbs_in_sabdab = list(db.db_summary.keys())

CDR_seqs, CDR_BB_coords = get_sabdab_fabs(all_pdbs_in_sabdab)

Filter FABs ━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  11% 0:01:17

KeyboardInterrupt: 

In [31]:
# with open('train_data/CDR_BB_coords.npy', 'wb') as outfile:
#    np.save(outfile, CDR_BB_coords)

# with open('train_data/CDR_seqs.npy', 'wb') as outfile:
#    np.save(outfile, CDR_seqs)

In [8]:
with open('train_data/CDR_BB_coords.npy', 'rb') as infile:
    CDR_BB_coords = np.load(infile, allow_pickle=True)

with open('train_data/CDR_seqs.npy', 'rb') as infile:
    CDR_seqs = np.load(infile, allow_pickle=True)

**1.2 Format data to model inputs**

Data loaded from SAbDab is reformated to model inputs and the training outputs. 

Each backbone atom corresponds to one node in the graph. The atoms are encoded into a vector with 41 elements (one-hot encoding of amino acid residue, one-hot encoding of atom type, one-hot encoding of CDR loop, positional encoding within loop).

The input coordinates of each backbone atoms are processed as follows. Anchor residues keep their original position, the CDR residues are spaced equally on a straigt line between the two anchors.

The training output coordinates correspond to the backbone coordinates from the crystal structure formated identically to the input coordinates

In [12]:
# functions that convert data extracted from SAbDab to model input

def encode(x, classes):
    '''
    One hot encodes a scalar x into a vector of length classes.
    This is the function used for Sequence encoding.
    '''
    one_hot = np.zeros(classes)
    one_hot[x] = 1

    return one_hot

def one_hot(num_list, classes=20):
    '''
    One hot encodes a 1D vector x.
    This is the function used for Sequence encoding.
    '''
    end_shape = (len(num_list), classes)
    finish = np.zeros(end_shape)
    for i in range(end_shape[0]):
        finish[i] = encode(num_list[i], classes)

    return finish

def which_loop(loop_seq, cdr):
    '''
    Adds a one-hot encoded vector to each node describing which CDR it belongs to.
    '''
    CDRs = ["H1", "H2", "H3", "L1", "L2", "L3", "Anchor"]
    loop = np.zeros((len(loop_seq), len(CDRs)))
    loop[:, -1] = 1
    loop[2:-2] = np.array([1.0 if cdr == x else 0.0 for x in (CDRs)])[None].repeat(len(loop_seq) - 4, axis=0)

    return loop

def positional_encoding(sequence, n=5):
    '''
    Gives the network information on how close each resdiue is to the anchors
    '''
    encs = []
    L = len(sequence)
    for i in range(n):
        encs.append(np.cos((2 ** i) * np.pi * np.arange(L) / L))
        encs.append(np.sin((2 ** i) * np.pi * np.arange(L) / L))

    return np.array(encs).transpose()

def res_to_atom(res_encoding, n_atoms=4):
    '''
    Adds a one-hot encoded vector to each node describing what atom type it is.
    '''
    out_shape = (res_encoding.shape[0], n_atoms, 41)
    atom_encoding = np.zeros(out_shape)

    for i in range(len(res_encoding)):
        for j in range(n_atoms):
            atom_encoding[i, j, 0:37] = res_encoding[i]
            # add one-hot encoding for atom type
            atom_encoding[i, j, 37:] = one_hot([j], classes=n_atoms) 

    return atom_encoding

def prepare_input_loop(CDR_coord, CDR_seq, CDR):
    '''
    Generates input features to be fed into the network for a single CDR
    '''
    CDR_input_coord = copy.deepcopy(CDR_coord)
    # put CDR residues equally spaced on straight line between anchor residues 
    CDR_input_coord[1:-1] = np.linspace(CDR_coord[1], CDR_coord[-2], len(CDR_coord) - 2)
    # CDR_input_coord = rearrange(torch.tensor(CDR_input_coords), "i a d -> () (i a) d").float()

    one_hot_encoding = one_hot(np.array([short2num[amino] for amino in CDR_seq]))
    loop = which_loop(CDR_seq, CDR)
    positional = positional_encoding(CDR_seq)
    res_encoding = np.concatenate([one_hot_encoding, positional, loop], axis=1)
    atom_encoding = res_to_atom(res_encoding)

    # encoding = res_to_atom(torch.tensor(np.concatenate([one_hot_encoding, positional, loop], axis=1)).float())
    # encoding = rearrange(encoding, "i a d -> () (i a) d")

    return CDR_input_coord, atom_encoding

def prepare_model_input(CDR_seq, CDR_BB_coord):
    '''
    Prepares model inputs for a single FAB
    '''
    encodings = []
    geomins = []
    
    for CDR in CDR_BB_coord:
        geom, encode = prepare_input_loop(CDR_BB_coord[CDR], CDR_seq[CDR], CDR)
        encodings.append(encode)
        geomins.append(geom)

    # concatenate encodings and geoms into single array
    encodings = np.concatenate(encodings, axis=0)
    geomins = np.concatenate(geomins, axis=0)
    # format to tensor
    encodings = torch.from_numpy(encodings)
    geomins = torch.from_numpy(geomins)
    # rearrange tensors that atoms in one residue are nolonger grouped
    encodings = rearrange(encodings, "i a d -> (i a) d")
    geomins = rearrange(geomins, "i a d -> (i a) d")

    return geomins, encodings

def prepare_model_inputs(CDR_seqs, CDR_BB_coords):
    '''
    Prepares model inputs for a list of FABs
    '''
    encodings = []
    geomins = []

    for i in track(range(len(CDR_seqs)), description='Preparing model inputs'):
        geom, encode = prepare_model_input(CDR_seqs[i], CDR_BB_coords[i])
        encodings.append(encode)
        geomins.append(geom)

    return geomins, encodings

def prepare_model_output(CDR_BB_coords):
    '''
    Prepares model outputs for training, formated identically to inputs
    '''
    geomouts = []
    for CDR_BB_coord in track(CDR_BB_coords, description='Preparing model outputs'):
        geomout = []
        for _, coords in CDR_BB_coord.items():
            geomout.append(coords)

        # concatenate geoms into single array
        geomout = np.concatenate(geomout, axis=0)
        # format to tensor
        geomout = torch.from_numpy(geomout)
        # rearrange tensor
        geomout = rearrange(geomout, "i a d -> (i a) d")

        geomouts.append(geomout)
    return geomouts

def concatenate_data(encodings, geomins, geomouts, masks):
    '''
    Puts encodings, geomins and geomouts into a single array.
    '''
    data = []
    for i in range(len(encodings)):
        # potentially change list to dict
        data.append({'encodings': encodings[i],
                     'geomins': geomins[i],
                     'geomouts': geomouts[i],
                     'mask': masks[i]})

    return data

CDR loops retrieve from SAbDab differ in numbers of residues. This leads to the node feature and coordinate tensors being of different sizes. When the model is trained in batches it cannot handle different formats of input tensors. 

To avoid this issues a mask is created. The mask is a vector of length 504 (which is the number of atoms in the longest Fab) and contains n_atoms * 1s and is filed up with 0s to 504. The coordinates and input node features are also padded to a size of 504 with 0s.

In [ ]:
# creat mask
def create_mask(node_encoding, mask_lenght=504):
    '''
    Function that creates a mask of a certain length with 1s where there is an atom and filled up with 0s
    '''
    mask = torch.zeros((mask_lenght))
    mask[:len(node_encoding)] = 1
    return mask

# padding
def pad_data_for_batch(tensor, out_lenght=504):
    '''
    Pads all tensors with 0 to the same number of nodes
    '''
    ndim = tensor.shape[1]
    outtensor = torch.zeros((out_lenght, ndim))
    outtensor[:tensor.shape[0], :] = tensor
    return outtensor

Format the data extracted in the previous step into the input tensors.

1. node_encodings: matrix of size n_atoms * 41, encoding node features
2. geomins: matrix of size n_atoms * 3, encoding the input coordinates of the CDR backbone residues (all residues equally spaced on a straight line between anchors)
3. geomouts: matrix of size n_atoms * 3, encoding the original coordinates of the CDR backbone residues

In [13]:
geomins, node_encodings = prepare_model_inputs(CDR_seqs, CDR_BB_coords)
geomouts = prepare_model_output(CDR_BB_coords)

Preparing model outputs ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00

Creates mask for each fab

In [14]:
masks = []
for node_encoding in node_encodings:
    mask = create_mask(node_encoding)
    masks.append(mask)

Pads the node features and geoms to a length of 504

In [15]:
geomins_padded = []
geomouts_padded = []
node_encodings_padded = []

for i in range(len(geomins)):
    geomins_padded.append(pad_data_for_batch(geomins[i]))
    geomouts_padded.append(pad_data_for_batch(geomouts[i]))
    node_encodings_padded.append(pad_data_for_batch(node_encodings[i]))

geomins = geomins_padded
geomouts = geomouts_padded
node_encodings = node_encodings_padded

Concatenate the all data into a single array, where each element corresponds to a fab. Within an element the data is saved in a dictionary with keys: 'encodings', 'geomins', 'geomouts' and 'mask'

In [16]:
data = concatenate_data(node_encodings, geomins, geomouts, masks)
len(data)

8117

**1.3 Prepare data for training**

In [17]:
# split 0.6, 0.2, 0.2 in train, test and validation sets
train, test = train_test_split(data, test_size=50, random_state=42,)
train, validation = train_test_split(train, test_size=100, random_state=42)

len(train), len(validation), len(test)

(7967, 100, 50)

In [18]:
batch_size = 32
train_dataloader = torch.utils.data.DataLoader(train, 
                                               batch_size=batch_size,   # Batch size
                                               num_workers=1,           # Number of cpu's allocated to load the data (recommended is 4/GPU)
                                               shuffle=True,            # Whether to randomly shuffle data
                                               pin_memory=True,         # Enables faster data transfer to CUDA-enabled GPUs (page-locked memory)
                                               )

val_dataloader = torch.utils.data.DataLoader(validation, 
                                             batch_size=batch_size,
                                             num_workers=1,
                                             shuffle=True,
                                             pin_memory=True,
                                             )

test_dataloader = torch.utils.data.DataLoader(test, 
                                              batch_size=batch_size,
                                              num_workers=1,
                                              shuffle=True,
                                              pin_memory=True,
                                              )

### 2. Implement the EGNN

**2.1 EGNN basic implementation**

This first implementation of the model does not use a mask. This cannot be trained in batches as the sizes of input tensors need to be identical within a batch. Also, this model should be used with the not padded data.

In [19]:
class EGNN(torch.nn.Module):
    '''
    Singel layer of an EGNN.
    '''
    def __init__(self, node_dim, message_dim=32):
        super().__init__()

        edge_input_dim = (node_dim * 2) + 1

        self.edge_mlp = torch.nn.Sequential(
            torch.nn.Linear(edge_input_dim, 2*edge_input_dim),
            torch.nn.SiLU(),
            torch.nn.Linear(2*edge_input_dim, message_dim),
            torch.nn.SiLU()
        )

        self.node_mlp = torch.nn.Sequential(
            torch.nn.Linear(node_dim + message_dim, 2*node_dim),
            torch.nn.SiLU(),
            torch.nn.Linear(2*node_dim, node_dim),
        )

        self.coors_mlp = torch.nn.Sequential(
            torch.nn.Linear(message_dim, 2*message_dim),
            torch.nn.SiLU(),
            torch.nn.Linear(2*message_dim, 1)
        )
    
    def forward(self, node_features, coordinates):                                                        # We pass in a mask that tells us what nodes to consider and which to ignore.
        rel_coors = rearrange(coordinates, 'b i d -> b i () d') - rearrange(coordinates, 'b j d -> b () j d')  
        rel_dist = (rel_coors ** 2).sum(dim=-1, keepdim=True)                                                  

        feats_j = rearrange(node_features, 'b j d -> b () j d')      
        feats_i = rearrange(node_features, 'b i d -> b i () d')
        feats_i, feats_j = torch.broadcast_tensors(feats_i, feats_j)

        edge_input = torch.cat((feats_i, feats_j, rel_dist), dim=-1)

        m_ij = self.edge_mlp(edge_input)

        coor_weights = self.coors_mlp(m_ij)                                                          # We multiply the predicted weight by the mask (masked residue pairs will have zero weight).
        coor_weights = rearrange(coor_weights, 'b i j () -> b i j')

        rel_coors_normed = rel_coors / rel_dist.clip(min = 1e-8)    

        coors_out = coordinates + torch.einsum('b i j, b i j c -> b i c', coor_weights, rel_coors_normed)  

        m_i = m_ij.sum(dim=-2)                                                                      # To average we divide over the length for each batch (length = sum(mask)).

        node_mlp_input = torch.cat((node_features, m_i), dim=-1)
        node_out = node_features + self.node_mlp(node_mlp_input)                            # We set the update for maked residues to zero. 

        return node_out, coors_out

class EGNNModel(torch.nn.Module):
    '''
    4 EGNN layers joined into one Model
    '''
    def __init__(self, node_dim, layers=4, message_dim=32):
        super().__init__()

        self.layers = torch.nn.ModuleList([EGNN(node_dim, message_dim = message_dim) for _ in range(layers)])   # Initialise as many EGNN layers as needed

    def forward(self, node_features, coordinates):

        for layer in self.layers:                                                                            
            node_features, coordinates = layer(node_features, coordinates)                                      # Update node features and coordinates for each layer in the model
        
        return node_features, coordinates

class DecoyGen(torch.nn.Module):
    '''
    5 EGNN models run in parallel.
    '''
    def __init__(self, dims_in=41, decoys=5, **kwargs):
        super().__init__()
        self.blocks = torch.nn.ModuleList([EGNNModel(node_dim=dims_in, **kwargs) for _ in range(decoys)])
        self.decoys = decoys

    def forward(self, node_features, coordinates):
        geoms = torch.zeros((self.decoys, *coordinates.shape[1:]), device=coordinates.device)

        for i, block in enumerate(self.blocks):
            geoms[i] = block(node_features, coordinates)[1] # only save geoms

        return geoms

**2.2 EGNN with mask to account for batches with different numbers of nodes**

This model uses the mask and can therefore be trained in batches. Model has to be trained on padded data.

In [20]:
class MaskEGNN(torch.nn.Module):
    '''
    Singel layer of an EGNN.
    '''
    def __init__(self, node_dim, message_dim=32):
        super().__init__()

        edge_input_dim = (node_dim * 2) + 1

        self.edge_mlp = torch.nn.Sequential(
            torch.nn.Linear(edge_input_dim, 2*edge_input_dim),
            torch.nn.SiLU(),
            torch.nn.Linear(2*edge_input_dim, message_dim),
            torch.nn.SiLU()
        )

        self.node_mlp = torch.nn.Sequential(
            torch.nn.Linear(node_dim + message_dim, 2*node_dim),
            torch.nn.SiLU(),
            torch.nn.Linear(2*node_dim, node_dim),
        )

        self.coors_mlp = torch.nn.Sequential(
            torch.nn.Linear(message_dim, 4*message_dim),
            torch.nn.SiLU(),
            torch.nn.Linear(4*message_dim, 1)
        )
    
    def forward(self, node_features, coordinates, mask):                                                        # We pass in a mask that tells us what nodes to consider and which to ignore.
        pair_mask = rearrange(mask, 'b j -> b () j ()') * rearrange(mask, 'b i -> b i () ()')
        
        rel_coors = rearrange(coordinates, 'b i d -> b i () d') - rearrange(coordinates, 'b j d -> b () j d')  
        rel_dist = (rel_coors ** 2).sum(dim=-1, keepdim=True)                                                  

        feats_j = rearrange(node_features, 'b j d -> b () j d')      
        feats_i = rearrange(node_features, 'b i d -> b i () d')
        feats_i, feats_j = torch.broadcast_tensors(feats_i, feats_j)

        edge_input = torch.cat((feats_i, feats_j, rel_dist), dim=-1)

        m_ij = self.edge_mlp(edge_input)

        coor_weights = pair_mask * self.coors_mlp(m_ij)                                                          # We multiply the predicted weight by the mask (masked residue pairs will have zero weight).
        coor_weights = rearrange(coor_weights, 'b i j () -> b i j')

        rel_coors_normed = rel_coors / rel_dist.clip(min = 1e-8)    

        coors_out = coordinates + torch.einsum('b i j, b i j c -> b i c', coor_weights, rel_coors_normed)  

        m_i = torch.einsum('b i d, b -> b i d', (pair_mask * m_ij).sum(dim=-2), 1/mask.sum(-1))                 # To average we divide over the length for each batch (length = sum(mask)).

        node_mlp_input = torch.cat((node_features, m_i), dim=-1)
        node_out = node_features + mask.unsqueeze(-1) * self.node_mlp(node_mlp_input)                             # We set the update for maked residues to zero. 

        return node_out, coors_out

class MaskEGNNModel(torch.nn.Module):
    '''
    4 EGNN layers joined into one Model
    '''
    def __init__(self, node_dim, layers=4, message_dim=32):
        super().__init__()

        self.layers = torch.nn.ModuleList([MaskEGNN(node_dim, message_dim = message_dim) for _ in range(layers)])   # Initialise as many EGNN layers as needed

    def forward(self, node_features, coordinates, mask):

        for layer in self.layers:                                                                            
            node_features, coordinates = layer(node_features, coordinates, mask)                                      # Update node features and coordinates for each layer in the model
        
        return node_features, coordinates

class MaskDecoyGen(torch.nn.Module):
    '''
    5 EGNN models run in parallel.
    '''
    def __init__(self, dims_in=41, decoys=5, **kwargs):
        super().__init__()
        self.blocks = torch.nn.ModuleList([MaskEGNNModel(node_dim=dims_in, **kwargs) for _ in range(decoys)])
        self.decoys = decoys

    def forward(self, node_features, coordinates, mask):
        geoms = torch.zeros((self.decoys, *coordinates.shape[1:]), device=coordinates.device)

        for i, block in enumerate(self.blocks):
            geoms[i] = block(node_features, coordinates, mask)[1] # only save geoms

        return geoms

### 3. Train model
**3.1 loss functions**

The model is initially trained with RMSD as a loss function only. In a second step it is finetune with RMSD and dist check.

In [21]:
# set loss functions
def rmsd(prediction, truth):
    dists = (prediction - truth).pow(2).sum(-1)
    return torch.sqrt(dists.mean(-1)).mean()

def rmsds(preds, true):
    return  torch.sort((preds - true).pow(2).sum(-1).mean(-1).pow(1/2))[0]

def length_penalty(pred):
    return ((((pred[:,1:]-pred[:,:-1])**2).sum(-1).pow(1/2) - 3.802).pow(2)).mean()

def different_penalty(pred):
    return -(rearrange(pred, "i n d -> i () n d") - rearrange(pred, "j n d -> () j n d")).pow(2).mean()

def dist_check(pred, amino):
    err = 0
    for i in range(6):
        CDR = rearrange(pred[:,amino[0,:,30+i]==1.0], "d (r a) p -> d a r p", a = 4)
        # CA-CA
        err += (((CDR[:,0,1:] - CDR[:,0,:-1]).pow(2).sum(-1).pow(1/2) - 3.82).abs() - 0.12).clamp(0).mean()
        # CA-N
        err += (((CDR[:,0] - CDR[:,1]).pow(2).sum(-1).pow(1/2) - 1.47).abs() - 0.01).clamp(0).mean()
        # CA-C
        err += (((CDR[:,0] - CDR[:,2]).pow(2).sum(-1).pow(1/2) - 1.53).abs() - 0.01).clamp(0).mean()
        # C-N
        err += (((CDR[:,2,:-1] - CDR[:,1,1:]).pow(2).sum(-1).pow(1/2) - 1.34).abs() - 0.01).clamp(0).mean()
        # CA-CB
        CDR2 = rearrange(pred[:,(amino[0,:,30+i]==1.0) & (amino[0,:,5] != 1.0)], "d (r a) p -> d a r p", a = 4)
        err += (((CDR2[:,0] - CDR2[:,-1]).pow(2).sum(-1).pow(1/2) - 1.54).abs() - 0.01).clamp(0).mean()

    return err

def atom_dist(geom):
    return ((geom[:,None] - geom[:,:,None]).mean(0).pow(2).sum(-1) + 1e-8).pow(1/2) 

def atom_dist_penal(geom, pred):
    true_ds = atom_dist(geom)
    pred_ds = atom_dist(pred)
    mask = true_ds < 4.0
    return (true_ds-pred_ds)[mask].pow(2).mean()

**3.2 Training loop**

Use this code to train the model without mask

In [22]:
# train for one epoch
def run_epoch(model, optim, train_dataloader, val_dataloader, grad_clip=10.0):
    '''
    Function to train model for a single epoch
    '''
    epoch_train_losses = []
    epoch_val_losses = []
    model.train()                                                      # Set the model to train mode (Should't matter here as we don't have dropout, but good practice to keep in)

    for i,data in enumerate(train_dataloader):                         # For each batch of data in the dataset
        coordinates, geomouts, node_features = data['geomins'].float(), data['geomouts'].float(), data['encodings'].float()

        pred = model(node_features, coordinates)
        optim.zero_grad()                                              # Delete old gradients

        loss = rmsd(geomouts, pred)
        epoch_train_losses.append(loss.item())                         # Store value of loss function for training set

        loss.backward()                                                # Calculate loss gradients (pytorch handles this in the background)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)  # Optional: Clip the norm of the gradient (It stops the optimiser from doing very large updates at once)
        optim.step()                                                   # Update model weights

    with torch.no_grad():                                              # Calculate loss funtion for validation set
        model.eval()                                                   # Set the model to eval mode
        for i, data in enumerate(val_dataloader):
            coordinates, geomouts, node_features, mask = data['geomins'].float(), data['geomouts'].float(), data['encodings'].float(), data['mask'].float()
            pred = model(node_features, coordinates, mask)
            loss = rmsd(geomouts, pred)
            epoch_val_losses.append(loss.item())
    
    return np.mean(epoch_train_losses), np.mean(epoch_val_losses)

Use this code to train the model with mask

In [23]:
# train for one epoch with mask
def mask_run_epoch(model, optim, train_dataloader, val_dataloader, grad_clip=10.0):
    '''
    Function to train model for a single epoch with mask
    '''
    epoch_train_losses = []
    epoch_val_losses = []
    model.train()                                                      # Set the model to train mode (Should't matter here as we don't have dropout, but good practice to keep in)

    for i,data in enumerate(train_dataloader):                         # For each batch of data in the dataset
        coordinates, geomouts, node_features, mask = data['geomins'].float(), data['geomouts'].float(), data['encodings'].float(), data['mask'].float()

        pred = model(node_features, coordinates, mask)
        optim.zero_grad()                                              # Delete old gradients

        loss = rmsd(geomouts, pred)
        epoch_train_losses.append(loss.item())                         # Store value of loss function for training set

        loss.backward()                                                # Calculate loss gradients (pytorch handles this in the background)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)  # Optional: Clip the norm of the gradient (It stops the optimiser from doing very large updates at once)
        optim.step()                                                   # Update model weights

    with torch.no_grad():                                              # Calculate loss funtion for validation set
        model.eval()                                                   # Set the model to eval mode
        for i, data in enumerate(val_dataloader):
            coordinates, geomouts, node_features, mask = data['geomins'].float(), data['geomouts'].float(), data['encodings'].float(), data['mask'].float()
            pred = model(node_features, coordinates, mask)
            loss = rmsd(geomouts, pred)
            epoch_val_losses.append(loss.item())
    
    return np.mean(epoch_train_losses), np.mean(epoch_val_losses)

In [24]:
def train_model(model, optimiser, train_dataloader, val_dataloader, n_epochs=5000, patience=150):
    '''
    Runs run_epoch function a specified number of times and keeps track of loss.
    '''
    train_losses = []
    val_losses = []

    print(" Train |  Val ")
    for epoch in track(range(n_epochs), description='Train model'):
        train_loss, val_loss = mask_run_epoch(model, optimiser, train_dataloader, val_dataloader)  # Run one epoch and get train and validation loss

        train_losses.append(train_loss)                                                    # Store train and validation loss
        val_losses.append(val_loss)

        if np.min(val_losses) == val_loss:                                                 # If it is the best model on the validation set, save it
            torch.save(model.state_dict(), "best_model")                                   # This is how you save models in pytorch
            epochs_without_improvement = 0

        elif epochs_without_improvement < patience:                                        # If the model hasn't improved this epoch store that
            epochs_without_improvement += 1
        else:                                                                              # If the model hasn't improved in 'patience' epochs stop the training.
            break

        if train_loss > 1.5*np.min(train_losses):                                          # EGNNs are quite unstable, this reverts the model to a previous state if an epoch blows up
            model.load_state_dict(torch.load("previous_weights", map_location=torch.device(device)))
            optimiser.load_state_dict(torch.load("previous_optim", map_location=torch.device(device)))
        if train_loss == np.min(train_losses):
            torch.save(model.state_dict(), "previous_weights")        
            torch.save(optimiser.state_dict(), "previous_optim")  


        print("{:6.2f} | {:6.2f}".format(train_loss, train_loss))
    
    return train_losses, val_losses

Code below actually trains the model

In [249]:
# torch settings
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_default_dtype(torch.float)

# initialise model
model = MaskDecoyGen().to(device = device).float()

# set optimiser
optimiser = torch.optim.RAdam(model.parameters(), lr=1e-3, weight_decay=1e-3)

# Step to actually train the network
train_losses, val_losses = train_model(model, optimiser, train_dataloader, val_dataloader, n_epochs=5000, patience=150)

Train model ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

Train model ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

Train model ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

Train model ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

Train model ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

...

Train model ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

Train model ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

Train model ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

Train model ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

Train model ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% -:--:--

In [ ]:
# export losses
losses = {'train_loss': train_losses,
          'val_loss': val_losses}

with open('train_data/losses.json', 'w') as outfile:
    json.dump(losses, outfile)

**3.3 Analyse trainig losses**

In [ ]:
import matplotlib.pyplot as plt
plt.figure(dpi=200)
plt.plot(train_losses[10:])        # I don't like to plot the first 10 epochs because they make it harder to see the rest of the data 
plt.plot(val_losses[10:])
plt.legend(["Train", "Val"])
plt.title("Evolution of loss during training")

### 4. Use Pytorch lighting for training

In [26]:
import pytorch_lightning
from pytorch_lightning.loggers.neptune import NeptuneLogger

In [27]:
class pl_EGNNModel(pytorch_lightning.LightningModule):
    def __init__(self):
        super().__init__()
        self.egnnmodel = MaskDecoyGen()

    def forward(self, node_encodings, coordinates, mask):

        return self.egnnmodel(node_encodings, coordinates, mask)   

    def configure_optimizers(self):
        optimizer = torch.optim.RAdam(model.parameters(), lr=1e-3, weight_decay=1e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        predicted_coordinates = self(batch['encodings'], batch['geomins'], batch['mask'])  
        loss = rmsd(batch['geomouts'], predicted_coordinates)
        return loss

    def validation_step(self, batch, batch_idx): 
        predicted_coordinates = self(batch['encodings'], batch['geomins'], batch['mask'])
        loss = rmsd(batch['geomouts'],  predicted_coordinates)
        return loss

    def validation_epoch_end(self, val_step_outputs): # Updated once when validation is called
        val_loss = torch.stack(val_step_outputs).detach().cpu().numpy().mean()
        self.logger.experiment['evaluation/val_loss'].log(val_loss)

In [28]:
ourlogger = NeptuneLogger(api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIyMGI0ZTUzYy0zMTBkLTRjMWMtODhjNS0wNTJmNjA1MzhmOGMifQ==",
              project="fspoendlin/ABlooper",
              name="Fabian",
              log_model_checkpoints=False,
              )

trainer = pytorch_lightning.Trainer(
    accelerator="auto",  # 'cpu' or 'gpu'
    max_epochs=5000,
    check_val_every_n_epoch=1,
    accumulate_grad_batches=None,
    gradient_clip_val=1.0,
    logger=ourlogger,
    ) 

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [29]:
model = pl_EGNNModel()
trainer.fit(model, train_dataloader, val_dataloader)


  | Name      | Type         | Params
-------------------------------------------
0 | egnnmodel | MaskDecoyGen | 662 K 
-------------------------------------------
662 K     Trainable params
0         Non-trainable params
662 K     Total params
2.649     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/fabian/miniconda3/envs/auto-db-pipeline/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:486: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
/Users/fabian/miniconda3/envs/auto-db-pipeline/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:01<?, ?it/s]